In [2]:

from dotenv import load_dotenv
import duckdb
from pathlib import Path

# testing
load_dotenv()

True

In [23]:
base_path = Path().cwd().parent
source_path = base_path / Path('silver/anilist/character/character-2024-1-17.parquet')
target_path = base_path / Path('gold/anilist/dim-character.parquet')

In [4]:
tb = duckdb.read_parquet(str(source_path))

In [24]:
tb_dim = duckdb.sql("""
WITH tb_filter_col AS (
    SELECT
        character_id
        , name_first
        , name_middle
        , name_last
        , name_full
        , name_native
        , name_alternative
        , image
        , description
        , gender
        , make_date(dateOfBirth_year, dateOfBirth_month, dateOfBirth_day) AS dateOfBirth
        , age
        , bloodType
        , siteUrl
    FROM tb
), dedupe AS (
    SELECT
        character_id
        , max(name_first) AS name_first
        , max(name_middle) AS name_middle
        , max(name_last) AS name_last
        , max(name_full) AS name_full
        , max(name_native) AS name_native
        , max(name_alternative) AS name_alternative
        , max(image) AS image
        , max(description) AS description
        , max(gender) AS gender
        , max(dateOfBirth) AS date_of_birth
        , max(age) AS age
        , max(bloodType) AS blood_type
        , max(siteUrl) AS site_url
        , current_date AS last_updated
    FROM tb_filter_col
    GROUP BY (character_id )
)
SELECT *
FROM dedupe
""")

In [25]:
tb_dim.to_parquet(str(target_path))